In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from collections import Counter
%matplotlib inline
from pandas import DataFrame
from pandas import Series
from pandas import concat
from pandas import read_csv
from pandas import datetime
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras import losses
from math import sqrt

In [2]:
train = pd.read_csv('../input/train_1.csv').fillna(0)
train1 = train.transpose()

df = train1
df.columns = df.iloc[0]
df = df[1:]

y=[]
for i in range(1,df.shape[1]):
    l = df.iloc[0:,i]
    y.append(list(map(int,l)))

In [3]:
import keras.backend as K
def smape_loss(y_true, y_pred):
    diff = K.abs((y_true - y_pred) / K.clip(K.abs(y_true) + K.abs(y_pred), K.epsilon(), None))
    return 200 * K.mean(diff, axis=-1)

In [4]:
import keras
import tensorflow as tf
%matplotlib inline
import os, sys
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [5]:
print("python:{}, keras:{}, tensorflow: {}".format(sys.version, keras.__version__, tf.__version__))

In [ ]:
def init():
    from keras.models import Sequential  
    from keras.layers.core import Dense, Activation  
    from keras.layers.recurrent import LSTM

    in_out_neurons = 50 
    hidden_neurons = 200

    model = Sequential()
    model.add(LSTM(hidden_neurons, return_sequences=False,
                   input_shape=(None, in_out_neurons)))
    model.add(Dense(in_out_neurons, input_dim=hidden_neurons))  
    model.add(Activation("linear"))  #softplus,softsign
    model.compile(loss=smape_loss, optimizer="Adam") #Adagrad
    return model

In [32]:
model=init()
model.summary()

In [14]:

import numpy as np

def _load_data(data, n_prev = 21):  
    """
    data should be pd.DataFrame()
    """

    docX, docY = [], []
    for i in range(len(data)-n_prev):
        docX.append(data.iloc[i:i+n_prev].as_matrix())
        docY.append(data.iloc[i+n_prev].as_matrix())
    alsX = np.array(docX)
    alsY = np.array(docY)

    return alsX, alsY

def train_test_split(df, test_size=0.2):  
    """
    This just splits data to training and testing parts
    """
    ntrn = round(len(df) * (1 - test_size))

    X_train, y_train = _load_data(df.iloc[0:ntrn])
    X_test, y_test = _load_data(df.iloc[ntrn:])

    return (X_train, y_train), (X_test, y_test)



In [68]:
# transform data to be stationary
raw_values = df.iloc[:,:50]

In [69]:
(X_train, y_train), (X_test, y_test)=train_test_split(raw_values)

In [20]:
X_train.shape,X_test.shape, y_train.shape,y_test.shape

In [70]:
model=init()
model.fit(X_train, y_train, batch_size=10, epochs=10, validation_split=0.05)

In [71]:
predicted = model.predict(X_test)
error =(((np.abs(y_test - predicted)) /(np.abs(y_test) + np.abs(predicted))) * 200).mean(axis=0)
error.mean()

In [75]:
series = []
for i in range(0,df.shape[1],50):
    series.append(i)
#sum(series)/len(series)

In [56]:
train_data =[]
for i in range(1,len(series)):
    raw_values = df.iloc[:,series[i-1]:series[i]]
    (X_train, y_train), (X_test, y_test)=train_test_split(raw_values)
    temp = list(X_train, y_train, X_test, y_test)
    train_data.append(temp)

In [78]:
models = []
result = []
model = init()

In [79]:
for i in range(1,len(series)):
    raw_values = df.iloc[:,series[i-1]:series[i]]
    (X_train, y_train), (X_test, y_test)=train_test_split(raw_values)
    model = init()
    model.fit(X_train, y_train, batch_size=10, epochs=10, validation_split=0.05)
    predicted = model.predict(X_test)
    error =(((np.abs(y_test - predicted)) /(np.abs(y_test) + np.abs(predicted))) * 200).mean(axis=0)
    result.append(error.mean())
    print(sum(result)/len(result))
    models.append(model)